In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
f = open('accessfiles.py', 'wb')
f.write(streaming_body_1.read())


import accessfiles

In [1]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8 MB 10.2 MB/s eta 0:00:01            | 3.6 MB 8.8 MB/s eta 0:00:257.5 MB/s eta 0:00:07█████████▏| 211.9 MB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 197 kB 55.2 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=93bffddfd0e252d9a68bbee8e9bd336aa6bfb67316298639117fe0cf7d02c446
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/01/c0/03/1c241c9c482b647d4d99412a98a5c7f87472728ad41ae55e1e
Successfully built pyspark


In [3]:
from zipfile import ZipFile
import time
import numpy as np
from operator import add

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

import os
from pyspark.sql.functions import unix_timestamp, to_date
import pyspark.sql.functions as F

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

from accessfiles import load_files

load_files()

In [40]:
from pyspark.sql import Row

movies_rdd=sc.textFile("movies.dat")
movie_nm=movies_rdd.map(lambda row:row.split("::")[1])
## get the year in the parenthesis
year=movie_nm.map(lambda row:row.split('(')[1].split(')')[0]).filter(lambda x: x.isdigit())
latest=year.max()
chk_str = "("+latest+")"
latest_mv_rdd=movie_nm.filter(lambda name: chk_str in name)
row = Row("Movies") # Column name
latest_mv_df = latest_mv_rdd.map(row).toDF().toPandas()

In [41]:
from accessfiles import create_download_link
create_download_link(latest_mv_df, filename='Latest_movies.csv')